In [1]:
## import
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score # Accuracy metrics 

2023-02-02 16:01:23.030359: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 16:01:23.723966: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/extras/CUPTI/lib64:/usr/local/cuda-11.2/lib64:
2023-02-02 16:01:23.726141: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/extras/CUPTI/lib64:/usr/local/cuda-11.2/l

In [ ]:
# model 2 
# I made it.
import tensorflow as tf

# 데이터 전처리
# 4개의 파일을 사용
data_forward_woman = np.loadtxt(fileUrl + '/forward/data_forward_woman.csv', unpack = True, delimiter=',', skiprows = 1, dtype=np.int64)
data_forward_man = np.loadtxt(fileUrl + '/forward/data_forward_man.csv', unpack = True, delimiter=',', skiprows = 1, dtype=np.int64)
data_left_woman = np.loadtxt(fileUrl + '/left/data_left_woman.csv', unpack = True, delimiter=',', skiprows = 1, dtype=np.int64)
data_left_man = np.loadtxt(fileUrl + '/left/data_left_man.csv', unpack = True, delimiter=',', skiprows = 1, dtype=np.int64)
data_right_woman = np.loadtxt(fileUrl + '/left/data_right_woman.csv', unpack = True, delimiter=',', skiprows = 1, dtype=np.int64)
data_right_man = np.loadtxt(fileUrl + '/left/data_right_man.csv', unpack = True, delimiter=',', skiprows = 1, dtype=np.int64)

# 두 명의 서 있는 자세와 앉아 있는 자세 병합
data_forward = np.concatenate((data_forward_woman, data_forward_man), axis = 1).transpose()
data_left = np.concatenate((data_left_woman, data_left_man), axis = 1).transpose()
data_right = np.concatenate((data_right_woman, data_right_man), axis = 1).transpose()

# 분류를 위한 라벨 구성  0= forward, 1=left, 2=right
label_forward = []
label_left = []
label_right = []
for i in range(len(data_forward)):
  label_forward.append(0)
for i in range(len(data_left)):
  label_left.append(1)
for i in range(len(data_right)):
  label_right.append(1)
    

# 앉아 있는 자세와 서 있는 자세에 대한 데이터를 합쳐서 데이터 전처리 마무리
data = np.concatenate((data_forward, data_left, data_right), axis = 0)
label = np.array(label_forward + label_left + label_right)


X_train, X_test, y_train, y_test = train_test_split(data, label, test_size = 0.2, random_state = 121)

print(X_train.shape)
print(y_train.shape)

# Architecture
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3))) #BGR or RGB
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax')) # normal, or alert

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on your training data
model.fit(X_train, y_train, epochs=10)

In [ ]:
# evaluate 2
results = model.evaluate(X_test, y_test)
print('accuracy: ', results[1])

In [ ]:
model.save('Maded_model.h5') #save